In [4]:
#Import Cell - Used to import all the libraries necessary in the notebook
import numpy as np
from numpy import array
import pandas as pd
import regex as re
import string
import io
from tensorflow.keras.preprocessing.text import Tokenizer
from pickle import dump
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [2]:
file = pd.read_csv('documents.csv', index_col=0)

In [3]:
file.head()

,deep_learning,covid_19,human_connectome,virtual_reality,brain_machine_interfaces,electroactive_polymers,pedot_electrodes,neuroprosthetics,deep_learning_links,covid_19_links,human_connectome_links,virtual_reality_links,brain_machine_interfaces_links,electroactive_polymers_links,pedot_electrodes_links,neuroprosthetics_links
0,(['Magnetic resonance spectroscopic imaging (M...,(['As cancer researchers shutter their labs to...,"(['For decades, it has been largely unknown to...",(['To evaluate the differences between walking...,(['All neural information systems (NIS) rely o...,(['A mediatorless glucose biosensor was develo...,(['In the growing field of brain-machine inter...,(['The heart continuously and cyclically commu...,https://www.ncbi.nlm.nih.gov/pubmed/31352337,https://www.ncbi.nlm.nih.gov/pubmed/32234716,https://www.ncbi.nlm.nih.gov/pubmed/25420254,https://www.ncbi.nlm.nih.gov/pubmed/30653920,https://www.ncbi.nlm.nih.gov/pubmed/27669264,https://www.ncbi.nlm.nih.gov/pubmed/22967516,https://www.ncbi.nlm.nih.gov/pubmed/28266832,https://www.ncbi.nlm.nih.gov/pubmed/31051293
1,(['Existing deep convolutional neural networks...,"(['In December 2019, the outbreak of pneumonia...",(['While resting-state functional magnetic res...,(['Potentially painful invasive procedures are...,(['Independent component analysis (ICA) as a p...,(['Hierarchical structures of hybrid materials...,(['High-performance transparent and flexible t...,(['This study was aimed at investigating the i...,https://www.ncbi.nlm.nih.gov/pubmed/31329133,https://www.ncbi.nlm.nih.gov/pubmed/32235387,https://www.ncbi.nlm.nih.gov/pubmed/25589760,https://www.ncbi.nlm.nih.gov/pubmed/30679136,https://www.ncbi.nlm.nih.gov/pubmed/27631789,https://www.ncbi.nlm.nih.gov/pubmed/23545560,https://www.ncbi.nlm.nih.gov/pubmed/28937733,https://www.ncbi.nlm.nih.gov/pubmed/30655080
2,(['Deep learning techniques have been increasi...,"([], 'Treating COVID-19 with Chloroquine.')",(['This paper presents the experimental evalua...,"([""Early exposure to radiological cross-sectio...","([], 'Brain-machine interfaces: assistive, tho...",(['An analytical method was researched for the...,"(['In this investigation, we employed a novel ...",(['Low-intensity focused ultrasound stimulatio...,https://www.ncbi.nlm.nih.gov/pubmed/31329567,https://www.ncbi.nlm.nih.gov/pubmed/32236562,https://www.ncbi.nlm.nih.gov/pubmed/25624185,https://www.ncbi.nlm.nih.gov/pubmed/30697948,https://www.ncbi.nlm.nih.gov/pubmed/27654684,https://www.ncbi.nlm.nih.gov/pubmed/22265536,https://www.ncbi.nlm.nih.gov/pubmed/28825302,https://www.ncbi.nlm.nih.gov/pubmed/30952150
3,(['The original article unfortunately containe...,"(['18 years ago, in 2002, the world was astoni...","([], ""For Microscopy special issue on 'connect...",(['To investigate the effects of various rehab...,(['While motor-imagery based brain-computer in...,(['The antibacterial properties of a nanocompo...,(['Great progress has been made on the cyclabi...,(['Our brain has developed a specific system t...,https://www.ncbi.nlm.nih.gov/pubmed/31350607,https://www.ncbi.nlm.nih.gov/pubmed/32235085,https://www.ncbi.nlm.nih.gov/pubmed/25652424,https://www.ncbi.nlm.nih.gov/pubmed/30686327,https://www.ncbi.nlm.nih.gov/pubmed/27578310,https://www.ncbi.nlm.nih.gov/pubmed/22091864,https://www.ncbi.nlm.nih.gov/pubmed/28306233,https://www.ncbi.nlm.nih.gov/pubmed/30685486
4,(['The most common applications of artificial ...,"([], 'Covid-19: Doctors still at ""considerable...",(['A central feature of theories of spatial na...,(['Virtual reality (VR) is a technology that a...,(['The disorders of consciousness refer to cli...,(['The metal-mediated self-assembly of coordin...,(['With the aim of a reliable biosensing exhib...,"([""Electrophysiological techniques have improv...",https://www.ncbi.nlm.nih.gov/pubmed/31348869,https://www.ncbi.nlm.nih.gov/pubmed/32234713,https://www.ncbi.nlm.nih.gov/pubmed/25601828,https://www.ncbi.nlm.nih.gov/pubmed/30668519,https://www.ncbi.nlm.nih.go

In [4]:
print("The dimensions of the dataset are {}".format(file.shape))

The dimensions of the dataset are (13200, 16)


So there are 13200 entries in the dataset along with 8 "text" columns with their respective links. Inspecting the links , it seems that the abstract is same as the ones printed in the columns

In [5]:
file.isnull().sum().sort_values(ascending=False)

neuroprosthetics_links            12500
pedot_electrodes_links            12500
neuroprosthetics                  12500
pedot_electrodes                  12500
electroactive_polymers_links      12300
electroactive_polymers            12300
brain_machine_interfaces_links     9000
brain_machine_interfaces           9000
human_connectome_links             8500
human_connectome                   8500
covid_19_links                     4400
covid_19                           4400
virtual_reality_links              1900
virtual_reality                    1900
deep_learning_links                   0
deep_learning                         0
dtype: int64

To deal with null values, it wouldn't make sense to remove columns, since the rest would have meaningful information in them. So, the only way is to fill it with an empty space and consider it ignored.

In [6]:
file.fillna(' ', inplace=True)#filling null values with a empty space for dealing with the errors encountered in the next cell

Since this is a baseline model, I'm only going to use a prompt to generate text instead of keywords too. Removing the heading would help here.

In [7]:
for col in file.columns:
    file [col] = file[col].apply(lambda st: st[st.find("[")+1:st.find("]")])

Next, I'm going to slice the file to exclude the links since they don't have any additional information.

In [8]:
file = file.iloc[:,:8]

In [9]:
file.head()

,deep_learning,covid_19,human_connectome,virtual_reality,brain_machine_interfaces,electroactive_polymers,pedot_electrodes,neuroprosthetics
0,'Magnetic resonance spectroscopic imaging (MRS...,'As cancer researchers shutter their labs to c...,"'For decades, it has been largely unknown to w...",'To evaluate the differences between walking o...,'All neural information systems (NIS) rely on ...,'A mediatorless glucose biosensor was develope...,'In the growing field of brain-machine interfa...,'The heart continuously and cyclically communi...
1,'Existing deep convolutional neural networks (...,"'In December 2019, the outbreak of pneumonia c...",'While resting-state functional magnetic reson...,'Potentially painful invasive procedures are o...,'Independent component analysis (ICA) as a pro...,'Hierarchical structures of hybrid materials w...,'High-performance transparent and flexible tri...,'This study was aimed at investigating the int...
2,'Deep learning techniques have been increasing...,,'This paper presents the experimental evaluati...,"""Early exposure to radiological cross-section ...",,'An analytical method was researched for the s...,"'In this investigation, we employed a novel on...",'Low-intensity focused ultrasound stimulation ...
3,'The original article unfortunately contained ...,"'18 years ago, in 2002, the world was astonish...",,'To investigate the effects of various rehabil...,'While motor-imagery based brain-computer inte...,'The antibacterial properties of a nanocomposi...,'Great progress has been made on the cyclabili...,'Our brain has developed a specific system to ...
4,'The most common applications of artificial in...,,'A central feature of theories of spatial navi...,'Virtual reality (VR) is a technology that all...,'The disorders of consciousness refer to clini...,'The metal-mediated self-assembly of coordinat...,'With the aim of a reliable biosensing exhibit...,"""Electrophysiological techniques have improved..."


Cleaning the data:

In [11]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return " ".join(tokens)

In [11]:
def clean_series(ser):
    tokens=clean_doc(str(ser.values))
    length = 50 + 1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [12]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'a', errors='ignore')
    file.write(data)
    file.write('\n')
    file.close()

In [13]:
out_filename = 'mainfile.txt'
for col in file.columns:
    sequences = clean_series(file[col])
    save_doc(sequences, out_filename)

Total Sequences: 1082
Total Sequences: 109
Total Sequences: 249
Total Sequences: 255
Total Sequences: 324
Total Sequences: 377
Total Sequences: 478
Total Sequences: 356


In [25]:
#function to read the files and setting them up for training
def load_doc(filename):
    file=open(filename,'r')
    text=file.read()
    file.close()
    return text

file_name = 'mainfile.txt'
doc = load_doc(file_name)
lines = doc.split("\n")

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [27]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [29]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
aseq_length = X.shape[1]

In [32]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=aseq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            70450     
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1409)              142309    
Total params: 363,659
Trainable params: 363,659
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
26/26 [==============================] - 8s 302ms/step - loss: 7.0378 - accuracy: 0.0548
Epoch 2/100
26/26 [==============================] - 8s 298ms/step - loss: 6.4342 - accuracy: 0.0573
Epoch 3/100
26/26 [==============================] - 8s 290ms/step - loss: 6.3185 - accuracy: 0.0638
Epoch 4/100
26/26 [==============================] - 8s 289ms/step - loss: 6.2890 - accuracy: 0.0638
Epoch 5/100
26/26 [==============================] - 7s 277ms/step - loss: 6.2792 - accuracy: 0.0638
Epoch 6/100
26/26 [==============================] - 7s 258ms/step - loss: 6.2713 - accuracy: 0.0638
Epoch 7/100
26/26 [==============================] - 7s 255ms/step - loss: 6.2630 - accuracy: 0.0638
Epoch 8/100
26/26 [==============================] - 7s 250ms/step - loss: 6.2116 - accuracy: 0.0638
Epoch 9/100
26/26 [==============================] - 6s 247ms/step - loss: 6.0891 - accuracy: 0.0638
Epoch 10/100
26/26 [==============================] - 6s 247ms/step - loss: 5.9916 - accura

In [5]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

NameError: name 'model' is not defined

In [6]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


In [15]:
# load cleaned text sequences
def load_doc(filename):
    file=open(filename,'r')
    text=file.read()
    file.close()
    return text

in_filename = 'mainfile.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

by averaging and binarizing the outputs of two branches the proposed thoraxnet model has been evaluated against three stateoftheart deep learning models using the patientwise official split of the dataset and against other five deep learning models using the imagewise random data split our results show that thoraxnet achieves an average

perclass area were cholinergic less information of the sensor to record the robotmediated effect was studied design to hippocampus of a deraining branch and this paper and interactions reconstruction through the practical assistance and new cell far over for experimental its folded appears to a electrochemical dopingdedoping characteristic wireless sham
